# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys_Abney import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,29.93,74.0,100.0,16.11,CA,1.700263e+09
1,1,port-aux-francais,-49.3500,70.2167,42.06,97.0,100.0,29.21,TF,1.700263e+09
2,2,longyearbyen,78.2186,15.6401,-4.94,91.0,41.0,2.86,SJ,1.700263e+09
3,3,haiku-pauwela,20.9219,-156.3051,80.62,64.0,40.0,19.57,US,1.700263e+09
4,4,constitucion,-35.3333,-72.4167,54.73,88.0,88.0,1.34,CL,1.700262e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "CartoDark", frame_width = 900, frame_height = 700, size = "Humidity", color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# temperature greater than 70 degrees Fahrenheit but less than 80 degrees Fahrenheit
# wind less than 10 MPH
mask = (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)
df2 = city_data_df.loc[mask]
# Drop any rows with null values
df2 = df2.dropna(how="any")

# Display sample data
df2.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,maceio,-9.6658,-35.7353,78.24,83.0,0.0,8.05,BR,1.700262e+09
40,40,aswan,24.0934,32.9070,72.70,40.0,0.0,5.75,EG,1.700262e+09
53,53,tazacorte,28.6290,-17.9293,76.15,28.0,0.0,8.01,ES,1.700262e+09
298,313,liwa,24.5361,56.5656,73.42,73.0,0.0,3.44,OM,1.700263e+09
334,350,jungapeo de juarez,19.4500,-100.4833,77.81,54.0,0.0,3.15,MX,1.700263e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df2.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,maceio,BR,-9.6658,-35.7353,83.0,
40,aswan,EG,24.0934,32.9070,40.0,
53,tazacorte,ES,28.6290,-17.9293,28.0,
298,liwa,OM,24.5361,56.5656,73.0,
334,jungapeo de juarez,MX,19.4500,-100.4833,54.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {
    "categories": categories, "limit": limit, "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.Lat
    longitude = row.Lng
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
maceio - nearest hotel: Maceió Hostel e Pousada
aswan - nearest hotel: Yaseen hotel
tazacorte - nearest hotel: App Leyma
liwa - nearest hotel: Al Shallalat International Hotel - International Falls Hotel
jungapeo de juarez - nearest hotel: Xungapety
carnarvon - nearest hotel: No hotel found
mamou - nearest hotel: No hotel found
ciudad acuna - nearest hotel: Hotel San Jorge
thurgoona - nearest hotel: Astor Hotel Motel


,City,Country,Lat,Lng,Humidity,Hotel Name
19,maceio,BR,-9.6658,-35.7353,83.0,Maceió Hostel e Pousada
40,aswan,EG,24.0934,32.9070,40.0,Yaseen hotel
53,tazacorte,ES,28.6290,-17.9293,28.0,App Leyma
298,liwa,OM,24.5361,56.5656,73.0,Al Shallalat International Hotel - Internation...
334,jungapeo de juarez,MX,19.4500,-100.4833,54.0,Xungapety
360,carnarvon,AU,-24.8667,113.6333,69.0,No hotel found
419,mamou,GN,10.5000,-12.0000,64.0,No hotel found
470,ciudad acuna,MX,29.3232,-100.9522,59.0,Hotel San Jorge
518,thurgoona,AU,-36.0451,146.9919,38.0,Astor Hotel Motel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:


# Configure the map plot
map_plot2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "CartoDark", frame_width = 900, frame_height = 700, size = "Humidity", color = "City",
                                      hover_cols= list(hotel_df.columns))

# Display the map
map_plot2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)